## Compute HCF by year and safe into new file
Built to run on Casper where data is directly, rather than tranferring to local machine.

In [1]:
# Import libraries 
import comet as cm 
import numpy as np 
import xarray as xr 
import pickle
import pandas as pd
import datetime 
import datetime 
import time 
from ComputeHCF import HCF 
from joblib import Parallel, delayed 

# Plotting utils 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.util


## 1. Read in data

<b>What data do we need?</b> <br>
Vertical profiles of: temperature (T), specific humidity (Q), geopotential height (zg in CESM2, Z3 in other runs), and pressure (P). <br>
In addition, need lowest level temperature, specfic humidity, height, and pressure - so basically T2m, Q2m, PSfc, and 2m height. <br><br>
<b>Units:</b><br>
Temperature --> K <br>
Height      --> m <br>
Sp. Humidity -> kg/kg <br>
Pressure    --> Pa  <br><br>




In [ ]:
yearString = '1983'

### 1.1 Read in data for CAM6 + CLM5

In [ ]:
# Set up strings used to define/access each file 
dataDir    = '/glade/work/mdfowler/data/HighOutput_IslasSims/'

fileStart  = 'f.e21.FHIST.f09_f09.cesm2_cam6_clm5.001.cam.h4.'
profileEnd = '_conusAllTimes-VertProfiles.nc'
pressEnd   = '_conus-Pressure-UTCtimes.nc'

# Set up file names 
fileName_profile = dataDir+fileStart+yearString+profileEnd
fileName_press   = dataDir+fileStart+yearString+pressEnd

# Open files and save to larger arrays 
with xr.open_dataset(fileName_profile, decode_times=True) as fullDS_profiles:
    fullDS_profiles['time'] = fullDS_profiles.indexes['time'].to_datetimeindex()

with xr.open_dataset(fileName_press, decode_times=True) as fullDS_pressure:
    fullDS_pressure['time'] = fullDS_pressure.indexes['time'].to_datetimeindex()

print('Done with reading in files for year %s' % years[iYr])


In [ ]:
ds_CLM5 = fullDS_profiles
ds_CLM5['PRESSURE'] = (('time','lev','lat','lon'), fullDS_pressure.PRESSURE)
ds_CLM5['UTC_hr'] = (('time'), fullDS_pressure.UTC_hr)
ds_CLM5['UTC_day'] = (('time'), fullDS_pressure.UTC_day)
ds_CLM5['UTC_mon'] = (('time'), fullDS_pressure.UTC_mon)
ds_CLM5['UTC_yr'] = (('time'), fullDS_pressure.UTC_yr)


In [ ]:
# Pick out 12 UTC only (early morning for most of US)
ds_utc12_CLM5 = ds_CLM5.where( ds_CLM5.UTC_hr==12.0, drop=True )


In [ ]:
# Pick out JJA only 
ds_utc12_CLM5 = ds_utc12_CLM5.where( ds_utc12_CLM5.UTC_mon_CLM45>=6 , drop=True)
ds_utc12_CLM5 = ds_utc12_CLM5.where( ds_utc12_CLM5.UTC_mon_CLM45<=8 , drop=True)


In [ ]:
ds_utc12_CLM5

In [ ]:
# Isolate UTC times 
UTC_hr_CLM5  = ds_utc12_CLM5.UTC_hr.values
UTC_day_CLM5 = ds_utc12_CLM5.UTC_day.values
UTC_mon_CLM5 = ds_utc12_CLM5.UTC_mon.values
UTC_yr_CLM5  = ds_utc12_CLM5.UTC_yr.values

In [ ]:
# When converting to a dataframe, need to only have relevent dimension included in dataset. Otherwise,
#   the function to convert to DF will create extra entries along dimensions that things like temperature and 
#   humidity don't depend on, breaking the below HCF calculation cell. 

ds_utc12_CLM5 = ds_utc12_CLM5.drop('time_bnds')  # Want to get rid of nbnd dimension 

# Want to also drop ilev dimension
ds_utc12_CLM5 = ds_utc12_CLM5.drop('hyai')
ds_utc12_CLM5 = ds_utc12_CLM5.drop('hybi') 
ds_utc12_CLM5 = ds_utc12_CLM5.drop('ilev')

### 1.2 Read in data for CAM6 + CLM4.5

In [ ]:
# Set up strings used to define/access each file 
dataDir    = '/glade/work/mdfowler/data/HighOutput_IslasSims/'

fileStart  = 'f.e21.FHIST.f09_f09.cesm2_cam6_clm4p5.001.cam.h4.'
profileEnd = '_conusAllTimes-VertProfiles.nc'
pressEnd   = '_conus-Pressure-UTCtimes.nc'

# Set up file names 
fileName_profile = dataDir+fileStart+yearString+profileEnd
fileName_press   = dataDir+fileStart+yearString+pressEnd

# Open files and save
with xr.open_dataset(fileName_profile, decode_times=True) as fullDS_profiles:
    fullDS_profiles['time'] = fullDS_profiles.indexes['time'].to_datetimeindex()

with xr.open_dataset(fileName_press, decode_times=True) as fullDS_pressure:
    fullDS_pressure['time'] = fullDS_pressure.indexes['time'].to_datetimeindex()


print('Done with reading in files for year %s' % years[iYr])


In [ ]:
ds_CLM45 = fullDS_profiles
ds_CLM45['PRESSURE'] = (('time','lev','lat','lon'), fullDS_pressure.PRESSURE)
ds_CLM45['UTC_hr']   = (('time'), fullDS_pressure.UTC_hr)
ds_CLM45['UTC_day']  = (('time'), fullDS_pressure.UTC_day)
ds_CLM45['UTC_mon']  = (('time'), fullDS_pressure.UTC_mon)
ds_CLM45['UTC_yr']   = (('time'), fullDS_pressure.UTC_yr)


In [ ]:
# Pick out 12 UTC only (early morning for most of US)
ds_utc12_CLM45 = ds_CLM45.where( ds_CLM45.UTC_hr==12.0, drop=True )

In [ ]:
# Pick out JJA only 
ds_utc12_CLM45 = ds_utc12_CLM45.where( ds_utc12_CLM45.UTC_mon_CLM45>=6 , drop=True)
ds_utc12_CLM45 = ds_utc12_CLM45.where( ds_utc12_CLM45.UTC_mon_CLM45<=8 , drop=True)

In [ ]:
ds_utc12_CLM45

In [ ]:
# Isolate UTC times 
UTC_hr_CLM45  = ds_utc12_CLM45.UTC_hr.values
UTC_day_CLM45 = ds_utc12_CLM45.UTC_day.values
UTC_mon_CLM45 = ds_utc12_CLM45.UTC_mon.values
UTC_yr_CLM45  = ds_utc12_CLM45.UTC_yr.values

In [ ]:
# Also get lat and lon
lat = ds_utc12_CLM45.lat.values
lon = ds_utc12_CLM45.lon.values

In [ ]:
# When converting to a dataframe, need to only have relevent dimension included in dataset. Otherwise,
#   the function to convert to DF will create extra entries along dimensions that things like temperature and 
#   humidity don't depend on, breaking the below HCF calculation cell. 

ds_utc12_CLM45 = ds_utc12_CLM45.drop('time_bnds')  # Want to get rid of nbnd dimension 

# Want to also drop ilev dimension
ds_utc12_CLM45 = ds_utc12_CLM45.drop('hyai')
ds_utc12_CLM45 = ds_utc12_CLM45.drop('hybi') 
ds_utc12_CLM45 = ds_utc12_CLM45.drop('ilev')

## 2. Compute HCF

In [ ]:
# ------------------------------------------
# Compute for each day and lat/lon point
#   Takes 35-40 minutes to run for one year
# ------------------------------------------

# Define variable names 
Tname = 'T'
Qname = 'Q'
Zname = 'Z3'
Pname = 'PRESSURE'

# Number of levels to worry about in actual "sounding"
nLev  = len(ds_utc12_CLM45.lev)

# Define dimensions 
nLat  = len(ds_utc12_CLM45.lat)
nLon  = len(ds_utc12_CLM45.lon)
nTime = len(ds_utc12_CLM45.time)

# Define empty arrays to save things into 
TBM_CLM45      = np.full([nTime,nLat,nLon], np.nan)
BCLH_CLM45     = np.full([nTime,nLat,nLon], np.nan)
BCLP_CLM45     = np.full([nTime,nLat,nLon], np.nan)
TDEF_CLM45     = np.full([nTime,nLat,nLon], np.nan)
TRAN_H_CLM45   = np.full([nTime,nLat,nLon], np.nan)
TRAN_P_CLM45   = np.full([nTime,nLat,nLon], np.nan)
TRAN_T_CLM45   = np.full([nTime,nLat,nLon], np.nan)
SHDEF_M_CLM45  = np.full([nTime,nLat,nLon], np.nan)
LHDEF_M_CLM45  = np.full([nTime,nLat,nLon], np.nan)
EADV_M_CLM45   = np.full([nTime,nLat,nLon], np.nan)

TBM_CLM5       = np.full([nTime,nLat,nLon], np.nan)
BCLH_CLM5      = np.full([nTime,nLat,nLon], np.nan)
BCLP_CLM5      = np.full([nTime,nLat,nLon], np.nan)
TDEF_CLM5      = np.full([nTime,nLat,nLon], np.nan)
TRAN_H_CLM5    = np.full([nTime,nLat,nLon], np.nan)
TRAN_P_CLM5    = np.full([nTime,nLat,nLon], np.nan)
TRAN_T_CLM5    = np.full([nTime,nLat,nLon], np.nan)
SHDEF_M_CLM5   = np.full([nTime,nLat,nLon], np.nan)
LHDEF_M_CLM5   = np.full([nTime,nLat,nLon], np.nan)
EADV_M_CLM5    = np.full([nTime,nLat,nLon], np.nan)

# Time how long this takes... 
t_start     = time.time()

for iLat in range(nLat):
    for iLon in range(nLon):
        for iT in range(nTime):

            # Pick out specific point and time period 
            DF_CLM45 = ds_utc12_CLM45.isel(lat=iLat,lon=iLon,time=iT).to_dataframe()
            DF_CLM5  = ds_utc12_CLM5.isel(lat=iLat,lon=iLon,time=iT).to_dataframe()

            # Flip order of levels so that surface comes first (required for function)
            DF_CLM45 = DF_CLM45.reindex(index=DF.index[::-1])
            DF_CLM5  = DF_CLM5.reindex(index=DF.index[::-1])

            # Compute HCF variables - CLM4.5
            TBM_CLM45[iT,iLat,iLon],BCLH_CLM45[iT,iLat,iLon],BCLP_CLM45[iT,iLat,iLon],TDEF_CLM45[iT,iLat,iLon],TRAN_H_CLM45[iT,iLat,iLon],TRAN_P_CLM45[iT,iLat,iLon],TRAN_T_CLM45[iT,iLat,iLon],SHDEF_M_CLM45[iT,iLat,iLon],LHDEF_M_CLM45[iT,iLat,iLon], EADV_M_CLM45[iT,iLat,iLon] = HCF(DF_CLM45, 
                                                                      Tname, 
                                                                      Qname, 
                                                                      Zname, 
                                                                      Pname, 
                                                                      nLev) 

            # Compute HCF variables - CLM5
            TBM_CLM5[iT,iLat,iLon],BCLH_CLM5[iT,iLat,iLon],BCLP_CLM5[iT,iLat,iLon],TDEF_CLM5[iT,iLat,iLon],TRAN_H_CLM5[iT,iLat,iLon],TRAN_P_CLM5[iT,iLat,iLon],TRAN_T_CLM5[iT,iLat,iLon],SHDEF_M_CLM5[iT,iLat,iLon],LHDEF_M_CLM5[iT,iLat,iLon], EADV_M_CLM5[iT,iLat,iLon] = HCF(DF_CLM5, 
                                                                      Tname, 
                                                                      Qname, 
                                                                      Zname, 
                                                                      Pname, 
                                                                      nLev) 


            
    print('Done with lat %i of %i ' % (iLat, nLat))


print('Time elapsed for all points and times: %.3f sec' % (time.time() - t_start))


## 3. Create dataset and save to netCDF

In [ ]:
# ---------------------------------------
# Create xr dataset from variables above 
# ---------------------------------------

# First set missing values to -9999
missingValue  = -9999

TBM_writeCLM45     = np.copy(TBM_CLM45)
BCLH_writeCLM45    = np.copy(BCLH_CLM45)
BCLP_writeCLM45    = np.copy(BCLP_CLM45)
TDEF_writeCLM45    = np.copy(TDEF_CLM45)
TRAN_H_writeCLM45  = np.copy(TRAN_H_CLM45)
TRAN_P_writeCLM45  = np.copy(TRAN_P_CLM45)
TRAN_T_writeCLM45  = np.copy(TRAN_T_CLM45)
SHDEF_M_writeCLM45 = np.copy(SHDEF_M_CLM45)
LHDEF_M_writeCLM45 = np.copy(LHDEF_M_CLM45)
EADV_M_writeCLM45  = np.copy(EADV_M_CLM45)

TBM_writeCLM45    [np.isnan(TBM_CLM45)    ==True] = missingValue
BCLH_writeCLM45   [np.isnan(BCLH_CLM45)   ==True] = missingValue
BCLP_writeCLM45   [np.isnan(BCLP_CLM45)   ==True] = missingValue
TDEF_writeCLM45   [np.isnan(TDEF_CLM45)   ==True] = missingValue
TRAN_H_writeCLM45 [np.isnan(TRAN_H_CLM45) ==True] = missingValue
TRAN_P_writeCLM45 [np.isnan(TRAN_P_CLM45) ==True] = missingValue
TRAN_T_writeCLM45 [np.isnan(TRAN_T_CLM45) ==True] = missingValue
SHDEF_M_writeCLM45[np.isnan(SHDEF_M_CLM45)==True] = missingValue
LHDEF_M_writeCLM45[np.isnan(LHDEF_M_CLM45)==True] = missingValue
EADV_M_writeCLM45 [np.isnan(EADV_M_CLM45) ==True] = missingValue
 
HCF_ds_CLM45 = xr.Dataset({
    'TBM': xr.DataArray(
                data   = TBM_writeCLM45,   # enter data here
                dims   = ['time','lat','lon'],
                coords = {'time': ds_utc12_CLM45.time.values, 'lat':lat, 'lon': lon},
                attrs  = {
                    '_FillValue': missingValue,
                    'units'     : 'K',
                    'LongName'  : 'Buoyant mixing potential temperature (convective threshold)'
                    }
                ),
    'BCLH': xr.DataArray(
                data   = BCLH_writeCLM45,   # enter data here
                dims   = ['time','lat','lon'],
                coords = {'time': ds_utc12_CLM45.time.values, 'lat':lat, 'lon': lon},
                attrs  = {
                    '_FillValue': missingValue,
                    'units'     : 'm',
                    'LongName'  : 'Height above ground of convective threshold'
                    }
                ),
    'BCLP': xr.DataArray(
            data   = BCLP_writeCLM45,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM45.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'Pa',
                'LongName'  : 'Pressure of convective threshold'
                }
            ),
    'TDEF': xr.DataArray(
            data   = TDEF_writeCLM45,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM45.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'K',
                'LongName'  : 'Potential temperature deficit needed to initiate convection'
                }
            ),
    'TRAN_H': xr.DataArray(
            data   = TRAN_H_writeCLM45,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM45.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'm',
                'LongName'  : 'Energy transition height'
                }
            ),
    'TRAN_P': xr.DataArray(
            data   = TRAN_P_writeCLM45,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM45.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'Pa',
                'LongName'  : 'Energy transition pressure'
                }
            ),
    'TRAN_T': xr.DataArray(
            data   = TRAN_T_writeCLM45,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM45.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'K',
                'LongName'  : 'Energy transition temperature'
                }
            ),
    'SHDEF_M': xr.DataArray(
            data   = SHDEF_M_writeCLM45,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM45.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'J/m2',
                'LongName'  : 'Sensible heat deficit of mixed layer'
                }
            ),
    'LHDEF_M': xr.DataArray(
            data   = LHDEF_M_writeCLM45,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM45.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'J/m2',
                'LongName'  : 'Latent heat deficit of mixed layer'
                }
            ),
    'EADV_M': xr.DataArray(
            data   = EADV_M_writeCLM45,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM45.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : '-',
                'LongName'  : 'Energy advantage of mixed layer'
                }
            )
    } )

# -------------------
# Save to netCDF 
# -------------------

savePath = '/glade/work/mdfowler/scripts/Coupling_CAM6CLM5/processed_data/HCFvariables-CAM6CLM4p5_'+yearString+'.nc'
HCF_ds_CLM45.to_netcdf(savePath, mode='w')


In [ ]:
# ---------------------------------------
# Create xr dataset from variables above 
# ---------------------------------------

# First set missing values to -9999
missingValue  = -9999

TBM_writeCLM5     = np.copy(TBM_CLM5)
BCLH_writeCLM5    = np.copy(BCLH_CLM5)
BCLP_writeCLM5    = np.copy(BCLP_CLM5)
TDEF_writeCLM5    = np.copy(TDEF_CLM5)
TRAN_H_writeCLM5  = np.copy(TRAN_H_CLM5)
TRAN_P_writeCLM5  = np.copy(TRAN_P_CLM5)
TRAN_T_writeCLM5  = np.copy(TRAN_T_CLM5)
SHDEF_M_writeCLM5 = np.copy(SHDEF_M_CLM5)
LHDEF_M_writeCLM5 = np.copy(LHDEF_M_CLM5)
EADV_M_writeCLM5  = np.copy(EADV_M_CLM5)

TBM_writeCLM5    [np.isnan(TBM_CLM5)    ==True] = missingValue
BCLH_writeCLM5   [np.isnan(BCLH_CLM5)   ==True] = missingValue
BCLP_writeCLM5   [np.isnan(BCLP_CLM5)   ==True] = missingValue
TDEF_writeCLM5   [np.isnan(TDEF_CLM5)   ==True] = missingValue
TRAN_H_writeCLM5 [np.isnan(TRAN_H_CLM5) ==True] = missingValue
TRAN_P_writeCLM5 [np.isnan(TRAN_P_CLM5) ==True] = missingValue
TRAN_T_writeCLM5 [np.isnan(TRAN_T_CLM5) ==True] = missingValue
SHDEF_M_writeCLM5[np.isnan(SHDEF_M_CLM5)==True] = missingValue
LHDEF_M_writeCLM5[np.isnan(LHDEF_M_CLM5)==True] = missingValue
EADV_M_writeCLM5 [np.isnan(EADV_M_CLM5) ==True] = missingValue
 
HCF_ds_CLM5 = xr.Dataset({
    'TBM': xr.DataArray(
                data   = TBM_writeCLM5,   # enter data here
                dims   = ['time','lat','lon'],
                coords = {'time': ds_utc12_CLM5.time.values, 'lat':lat, 'lon': lon},
                attrs  = {
                    '_FillValue': missingValue,
                    'units'     : 'K',
                    'LongName'  : 'Buoyant mixing potential temperature (convective threshold)'
                    }
                ),
    'BCLH': xr.DataArray(
                data   = BCLH_writeCLM5,   # enter data here
                dims   = ['time','lat','lon'],
                coords = {'time': ds_utc12_CLM5.time.values, 'lat':lat, 'lon': lon},
                attrs  = {
                    '_FillValue': missingValue,
                    'units'     : 'm',
                    'LongName'  : 'Height above ground of convective threshold'
                    }
                ),
    'BCLP': xr.DataArray(
            data   = BCLP_writeCLM5,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM5.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'Pa',
                'LongName'  : 'Pressure of convective threshold'
                }
            ),
    'TDEF': xr.DataArray(
            data   = TDEF_writeCLM5,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM5.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'K',
                'LongName'  : 'Potential temperature deficit needed to initiate convection'
                }
            ),
    'TRAN_H': xr.DataArray(
            data   = TRAN_H_writeCLM5,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM5.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'm',
                'LongName'  : 'Energy transition height'
                }
            ),
    'TRAN_P': xr.DataArray(
            data   = TRAN_P_writeCLM5,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM5.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'Pa',
                'LongName'  : 'Energy transition pressure'
                }
            ),
    'TRAN_T': xr.DataArray(
            data   = TRAN_T_writeCLM5,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM5.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'K',
                'LongName'  : 'Energy transition temperature'
                }
            ),
    'SHDEF_M': xr.DataArray(
            data   = SHDEF_M_writeCLM5,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM5.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'J/m2',
                'LongName'  : 'Sensible heat deficit of mixed layer'
                }
            ),
    'LHDEF_M': xr.DataArray(
            data   = LHDEF_M_writeCLM5,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM5.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : 'J/m2',
                'LongName'  : 'Latent heat deficit of mixed layer'
                }
            ),
    'EADV_M': xr.DataArray(
            data   = EADV_M_writeCLM5,   # enter data here
            dims   = ['time','lat','lon'],
            coords = {'time': ds_utc12_CLM5.time.values, 'lat':lat, 'lon': lon},
            attrs  = {
                '_FillValue': missingValue,
                'units'     : '-',
                'LongName'  : 'Energy advantage of mixed layer'
                }
            )
    } )

# -------------------
# Save to netCDF 
# -------------------

savePath = '/glade/work/mdfowler/scripts/Coupling_CAM6CLM5/processed_data/HCFvariables-CAM6CLM5_'+yearString+'.nc'
HCF_ds_CLM5.to_netcdf(savePath, mode='w')
